<a href="https://colab.research.google.com/github/sisifo3/P_T_2/blob/main/Net_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import argparse
import pandas as pd
import numpy as np
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import torch.optim as optim
import torch
import torch.nn as nn


In [12]:

class NoisyImages():
    def __init__(self,transform=None):
        #self.dataset = pd.read_csv(path_to_trainset, sep=' ')
        self.dataset = pd.read_csv('/content/drive/MyDrive/train_circle_v4/train_set_v2.csv', sep=' ')
        self.transform = transform

    def __getitem__(self, idx):
        image = np.load(self.dataset.iloc[idx, 0].split(',')[0])
        target = [self.dataset.iloc[idx, 0].split(',')[i] for i in range(1, 4)]
        
        if self.transform:
            image = np.expand_dims(np.asarray(image), axis=0)
            image = torch.from_numpy(np.array(image, dtype=np.float32))
            target = torch.from_numpy(np.array(np.asarray(target), dtype=np.float32))
            image = self.transform(image)

        return image, target

    def __len__(self):
        return len(self.dataset)


In [10]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
device = get_device()

In [4]:
import torch.nn as nn


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input images 1 * 200 * 200
        self.L1 = nn.Sequential(nn.Conv2d(1, 32, 5), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2, 2))  # 32 * 98 * 98
        self.L2 = nn.Sequential(nn.Conv2d(32, 64, 3), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2))  # 64 * 48 * 48
        self.L3 = nn.Sequential(nn.Conv2d(64, 128, 3), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2))  # 128 * 23 * 23
        self.L4 = nn.Sequential(nn.Conv2d(128, 128, 3), nn.BatchNorm2d(128), nn.ReLU())  # 128 * 21 * 21
        self.L5 = nn.Sequential(nn.Conv2d(128, 4, 1), nn.BatchNorm2d(4), nn.ReLU())  # 4 * 21 * 21
        self.FC = nn.Sequential(nn.Linear(4 * 21* 21, 256), nn.ReLU(), nn.Linear(256, 16), nn.ReLU())
        self.Last = nn.Linear(16, 3)

    def forward(self, x):
        x = self.L5(self.L4(self.L3(self.L2(self.L1(x)))))
        B, C, H, W = x.shape
        x = x.view(-1, C * H * W)
        x = self.FC(x)
        x = self.Last(x)
        return x

In [7]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate"""
    lr = 0.001
    if 20 < epoch <= 30:
        lr = 0.0001
    elif 30 < epoch :
        lr = 0.00001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    print("learning rate -> {}\n".format(lr))


In [ ]:
def main():

    model = Net()
    #model = nn.DataParallel(model)
    model.to(device)

    criterion1 = nn.MSELoss()
    criterion2 = nn.L1Loss()

    optimizer = optim.Adam(model.parameters(), lr =0.001)

    cudnn.benchmark = True
    normalize = transforms.Normalize(mean=[0.5], std=[0.5])

    trainset = NoisyImages(transforms.Compose([normalize,]))

    print(trainset)
    train_loader = torch.utils.data.DataLoader(
        trainset, batch_size= 4, shuffle=True,
        num_workers=2)

    #output = open(args.out_file, "w")
    for epoch in range(2):
        print('Epoch {}'.format(epoch))

        adjust_learning_rate(optimizer, epoch)
        model.train()
        running_loss = 0.0
        
        print(enumerate(train_loader))

        for i, (images, target) in enumerate(train_loader):

            images = images.to(device)
            target = target.to(device)

            output = model(images)

            loss = criterion1(output, target/200) + 0.1 * criterion2(output, target/200)

            # compute gradient and do optimizer step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            print('Loss: {:.4f}'.format(loss))
main()

Epoch 0
learning rate -> 0.001

Loss: 0.1372
Loss: 0.3246
Loss: 0.4597
Loss: 0.6933
Loss: 0.8136
Loss: 0.9329
Loss: 1.0724
Loss: 1.1903
Loss: 1.3443
Loss: 1.4483
Loss: 1.5135
Loss: 1.6687
Loss: 1.7404
Loss: 1.7903
Loss: 1.8980
Loss: 1.9909
Loss: 2.0811
Loss: 2.1474
Loss: 2.2337
Loss: 2.3230
Loss: 2.4355
Loss: 2.4948
Loss: 2.5886
Loss: 2.7038
Loss: 2.8224
Loss: 2.9209
Loss: 3.0667
Loss: 3.1545
Loss: 3.2371
Loss: 3.3251
Loss: 3.4527
Loss: 3.6130
Loss: 3.8348
Loss: 3.8651
Loss: 3.9646
Loss: 4.1556
Loss: 4.2199
Loss: 4.2912
Loss: 4.3701
Loss: 4.4504
Loss: 4.5991
Loss: 4.7071
Loss: 4.7640
Loss: 4.8392
Loss: 4.9098
Loss: 5.0009
Loss: 5.1159
Loss: 5.2961
Loss: 5.3793
Loss: 5.4651
Loss: 5.5612
Loss: 5.6348
Loss: 5.7184
Loss: 5.8179
Loss: 5.9953
Loss: 6.0867
Loss: 6.1515
Loss: 6.2384
Loss: 6.2969
Loss: 6.3772
Loss: 6.4797
Loss: 6.5494
Loss: 6.6508
Loss: 6.8125
Loss: 6.9199
Loss: 6.9827
Loss: 7.0879
Loss: 7.1801
Loss: 7.2720
Loss: 7.3993
Loss: 7.5013
Loss: 7.6355
Loss: 7.7165
Loss: 7.8231
Loss: 